### 03. RAG - Retrieval-Augmented Generation

- skicka med information som man önskar att LLM använder för att generera ett svar
- kan vara egen data som modell ej är tränad på eller data som är nyare än modellens träningsdata

- todo: maybe use vector db? https://www.youtube.com/watch?v=JfSmffOyV-8&t=488s
- todo: https://docs.trychroma.com/docs/overview/getting-started

![03 diagram](docs/03.drawio.png)

In [24]:
import os
from dotenv import load_dotenv
from pydantic import BaseModel

from google import genai  # client for google gemini api

In [25]:
load_dotenv()
API_KEY = os.getenv("GOOGLE_API_KEY")  # load the api key from another file (.env)

In [26]:
client = genai.Client(api_key=API_KEY)  # create an instance of the client

In [ ]:
response = client.models.generate_content(
    model="gemini-2.0-flash", contents="who won the masters in 2025?"
)

print(response.candidates[0].content.parts[0].text)

In [28]:
# provide information that can be used by the model

winners = None
with open("data/masters-winners.csv", encoding="utf-8") as f:
    winners = f.read()

In [ ]:
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=f"Use the following information to answer the question: {winners}. Who won the masters in 2025?",
)

# print the response
print(response.candidates[0].content.parts[0].text)

In [46]:
class WinnerResponse(BaseModel):
    winner: str | None
    year: int | None
    score: str | None

In [ ]:
# mixing provided data with the models training data is hard 🥲
# answer as text fails but when asking for a structured response if succeeds

prompt = """
    You are answering questions about the Masters golf tournament.
    Use your own knowledge first. Only refer to the following data if you're unsure or if your own answer is incomplete: {winners}

    Question: Who won the Masters in 1985 and what was the winning score?    
"""

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=prompt.strip(),
    config={
        "response_mime_type": "application/json",
        "response_schema": WinnerResponse,
    },  # define structured response
)

result = response.parsed
print(result.model_dump())

# print(response.candidates[0].content.parts[0].text)